In [1]:
!pip install mediapipe opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 32.3 MB/s eta 0:00:00


In [2]:
import cv2
import mediapipe as mp
import csv
import numpy as np

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [4]:
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    rows = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            coordinates = extract_coordinates(results)
            right_hip_ankle_angle = calculate_angle(coordinates['right_shoulder'], coordinates['right_hip'], coordinates['right_ankle'])
            left_hip_ankle_angle = calculate_angle(coordinates['left_shoulder'], coordinates['left_hip'], coordinates['left_ankle'])

            coordinates['right_hip_ankle_angle'] = right_hip_ankle_angle
            coordinates['left_hip_ankle_angle'] = left_hip_ankle_angle

            rows.append(coordinates)

    cap.release()

    save_to_csv(output_path, rows)

In [5]:
def extract_coordinates(results):
    landmarks = results.pose_landmarks.landmark

    coordinates = {
        "right_hip": (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y),
        "left_hip": (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y),
        "right_ankle": (landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y),
        "left_ankle": (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y),
        "left_shoulder": (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y),
        "right_shoulder": (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
    }

    return coordinates

In [6]:
def calculate_angle(a, b, c):
    a = np.array(a)  # 첫 번째 점
    b = np.array(b)  # 중간 점 (각도 점)
    c = np.array(c)  # 세 번째 점

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle

In [7]:
def save_to_csv(output_path, rows):
    with open(output_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["right_hip_x", "right_hip_y",
                         "left_hip_x", "left_hip_y",
                         "right_ankle_x", "right_ankle_y",
                         "left_ankle_x", "left_ankle_y",
                         "left_shoulder_x", "left_shoulder_y",
                         "right_shoulder_x", "right_shoulder_y",
                         "right_hip_ankle_angle", "left_hip_ankle_angle"])

        for row in rows:
            writer.writerow([row["right_hip"][0], row["right_hip"][1],
                             row["left_hip"][0], row["left_hip"][1],
                             row["right_ankle"][0], row["right_ankle"][1],
                             row["left_ankle"][0], row["left_ankle"][1],
                             row["left_shoulder"][0], row["left_shoulder"][1],
                             row["right_shoulder"][0], row["right_shoulder"][1],
                             row["right_hip_ankle_angle"], row["left_hip_ankle_angle"]])

In [8]:
if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/GDSC_AI/Solution Challenge/answer_vid.mp4"
    output_path = "SC_sample.csv"
    process_video(video_path, output_path)